# Capstone Project: IBM-Coursera Professional Data Science Specialization



Introduction

This project is designed to answer the question: Is there a San Francisco neighbourhood that has is a good location to open a Chinese restaurant.

This is my project notebook. I have also submitted a report on Github, and have posted this information to my blog.



# 1. Import, install and load.¶

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library


from bs4 import BeautifulSoup
import urllib.request


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [5]:
!pip install geopy
!conda update folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be UPDATED:

    ca-certificates: 2020.6.20-hecda079_0     conda-forge --> 2020.7.22-0      
    certifi:         2020.6.20-py36h9f0ad1d_0 conda-forge --> 2020.6.20-py36_0 

The following packages will be DOWNGRADED:

    openssl:         1.1.1g-h516909a_1        conda-forge --> 1.1.1g-h7b6447c_0

Preparing transaction: done
Verifying transaction: done
Executing transaction: done


# 2. Find data from the Web page containing San Francisco Zip codes, scrape into Jupyter notebook

In [6]:
#Gets the url and scrapes the html 
url = 'http://ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?ClientCode=capitolimpact&State=ca&StName=California&StFIPS=06&FIPS=06075'
req = urllib.request.urlopen(url)


soup = BeautifulSoup(req)

In [7]:
#Finds the table to scrape
table = soup.find('table')

#Provides the empty arrays for the html tags that are being grabbed and assigned to the headings
P = []
C = []

for row in table.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) == 3:
        P.append(cells[0].find(text=True))
        C.append(cells[1].find(text=True))

# 3. Merge Zip code numbers into City data

In [8]:
#Creates the dataframe and places the data in its respective columns
df_sanfran = pd.DataFrame(P, columns=['PostalCode'])
df_sanfran['City'] = C
df_sanfran.head(90)

,PostalCode,City
0,94101,San Francisco
1,94102,San Francisco
2,94103,San Francisco
3,94104,San Francisco
4,94105,San Francisco
5,94107,San Francisco
6,94108,San Francisco
7,94109,San Francisco
8,94110,San Francisco
9,94111,San Francisco


In [9]:
#A specialized function that joins the neighborhoods with the same postalcode
foo = lambda a: ','.join(a) 
df_sanfran = df_sanfran.groupby(['PostalCode']).agg({
                                'City': foo}).reset_index()

In [10]:
df_sanfran.head(48)

,PostalCode,City
0,94101,San Francisco
1,94102,San Francisco
2,94103,San Francisco
3,94104,San Francisco
4,94105,San Francisco
5,94107,San Francisco
6,94108,San Francisco
7,94109,San Francisco
8,94110,San Francisco
9,94111,San Francisco


# 4. Read the csv file containing latitude and longitude for each Zip code.

In [11]:
df_lonlat = pd.read_csv('https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/download/?format=csv&refine.state=CA&q=San+Francisco&timezone=America/Los_Angeles&lang=en&use_labels_for_header=true&csv_separator=%3B', delimiter=';')
df_lonlat.head(73)



,Zip,City,State,Latitude,Longitude,Timezone,Daylight savings time flag,geopoint
0,94138,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
1,94169,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
2,94153,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
3,94146,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
4,94165,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
5,94155,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
6,94080,South San Francisco,CA,37.652857,-122.430100,-8,1,"37.652857,-122.4301"
7,94175,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
8,94160,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"
9,94164,San Francisco,CA,37.784827,-122.727802,-8,1,"37.784827,-122.727802"


# 4A. Drop unnecessary columns¶

In [12]:
df_lonlat.drop(['City', 'State', 'Timezone', 'Daylight savings time flag', 'geopoint'], axis=1, inplace=True)
df_lonlat.head()

,Zip,Latitude,Longitude
0,94138,37.784827,-122.727802
1,94169,37.784827,-122.727802
2,94153,37.784827,-122.727802
3,94146,37.784827,-122.727802
4,94165,37.784827,-122.727802


In [13]:
df_lonlat.rename(columns={'Zip':'PostalCode'}, inplace=True)


In [14]:
df_sanfran.PostalCode = df_sanfran.PostalCode.astype(int)

# 5. Merge the Zip Code dataframe and the latitude / longitude dataframe.

In [15]:
df_sanfran = pd.merge(df_sanfran, df_lonlat, on='PostalCode', how='outer')


# 6. Generate map showing Zip codes.

In [16]:
# create map of San Francisco using latitude and longitude values
map_sanfran = folium.Map(location=[37.748730, -122.415450], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_sanfran['Latitude'], df_sanfran['Longitude'], df_sanfran['City']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sanfran)  
    
map_sanfran

# 6A. Scrape data from Web page listing San Francisco Zip codes.

In [17]:
#Gets the url and scrapes the html 
url1 = 'https://www.zipdatamaps.com/zipcodes-san-francisco-ca'
req1 = urllib.request.urlopen(url1)

soup1 = BeautifulSoup(req1)

# 6B. Load data into dataframe.

In [18]:
table1 = soup1.find('table', class_='table')

Post = []
Pop = []

for row in table1.find_all('tr'):
    cells = row.find_all('td')
    if len(cells) == 8:
        Post.append(cells[0].find(text=True))
        Pop.append(cells[5].find(text=True))

# 6C. Merge the Zip code, latitude, and longitude dataframes with population dataframe.

In [19]:
df_pop = pd.DataFrame(Post, columns=['PostalCode'])
df_pop['Population'] = Pop

In [20]:
df_pop.PostalCode = df_pop.PostalCode.astype(int)

In [21]:
df_merged = pd.merge(df_sanfran, df_pop, on='PostalCode', how='outer')


In [22]:
%%debug
cols = [2, 3, 4, 5, ]
df_merged.drop(df_merged.columns[cols], axis=0, inplace=True)


NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()

ipdb> c
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexes/base.py in __getitem__(self, key)
   3966 
   3967         key = com.values_from_object(key)
-> 3968         result = getitem(key)
   3969         if not is_scalar(result):
   3970             return promote(result)

IndexError: index 5 is out of bounds for axis 1 with size 5


In [23]:


df_merged = df_merged.dropna()

In [24]:
df_merged.head(70)

,PostalCode,City,Latitude,Longitude,Population
1,94102,San Francisco,37.779329,-122.41915,San Francisco
2,94103,San Francisco,37.772329,-122.41087,San Francisco
3,94104,San Francisco,37.791728,-122.40190,San Francisco
4,94105,San Francisco,37.789228,-122.39570,San Francisco
5,94107,San Francisco,37.766529,-122.39577,San Francisco
6,94108,San Francisco,37.792678,-122.40793,San Francisco
7,94109,San Francisco,37.792778,-122.42188,San Francisco
8,94110,San Francisco,37.748730,-122.41545,San Francisco
9,94111,San Francisco,37.798228,-122.40027,San Francisco
10,94112,San Francisco,37.720931,-122.44241,San Francisco


# 7. Submit Foursquare credentials¶

In [25]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: Y5G2ZFFUXD23IOJUOG4UATIMKMDQUJXLLXTAEYAHPTU2SZLL
CLIENT_SECRET:JFUCDQLXDX2E4ZP20EHH5JJ3H4L1QDJRT25ES14J0WA1C1VO


# 8. We look at all San Francisco Zip codes, using 94107 as a reference point

In [26]:
first_nei = df_merged['PostalCode'][5]
first_nei

94107

In [27]:
first_nei_lat = df_merged.loc[5,'Latitude']
first_nei_lon = df_merged.loc[5,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(first_nei, 
                                                               first_nei_lat, 
                                                               first_nei_lon))

Latitude and longitude values of 94107 are 37.766529, -122.39577.


In [28]:
radius = 500 
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    first_nei_lat, 
    first_nei_lon, 
    radius, 
    LIMIT)

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f605bc5bf4f711750d3057d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Showplace Square',
  'headerFullLocation': 'Showplace Square, San Francisco',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 28,
  'suggestedBounds': {'ne': {'lat': 37.7710290045, 'lng': -122.39008811633391},
   'sw': {'lat': 37.762028995499996, 'lng': -122.40145188366608}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '58d451f495383907dba37158',
       'name': 'Boba Guys',
       'location': {'address': '1002 16th St',
        'crossStreet': 'at Missouri St',
        'lat': 37.76644771569975,
        'lng': -122.39704236067308,
        'labeledLatLngs': 

In [30]:
results['response']['groups'][0]['items'][0]['venue']['categories'][0]['name']

'Bubble Tea Shop'

In [31]:
venues=results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.columns

Index(['reasons.count', 'reasons.items', 'referralId', 'venue.categories',
       'venue.delivery.id', 'venue.delivery.provider.icon.name',
       'venue.delivery.provider.icon.prefix',
       'venue.delivery.provider.icon.sizes', 'venue.delivery.provider.name',
       'venue.delivery.url', 'venue.id', 'venue.location.address',
       'venue.location.cc', 'venue.location.city', 'venue.location.country',
       'venue.location.crossStreet', 'venue.location.distance',
       'venue.location.formattedAddress', 'venue.location.labeledLatLngs',
       'venue.location.lat', 'venue.location.lng',
       'venue.location.neighborhood', 'venue.location.postalCode',
       'venue.location.state', 'venue.name', 'venue.photos.count',
       'venue.photos.groups', 'venue.venuePage.id'],
      dtype='object')

# 9. Next, we gather information about the venues in Zip Code 94114 from Foursquare.

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Boba Guys,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",37.766448,-122.397042
1,Bottom of the Hill,"[{'id': '4bf58dd8d48988d1e9931735', 'name': 'R...",37.765116,-122.396218
2,Fitness Urbano,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",37.765684,-122.397009
3,Pawtrero Hill Bathhouse and Feed Company,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",37.764140,-122.394500
4,UCSF Bakar Fitness & Rec Center,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",37.768146,-122.393290
5,Daggett Plaza,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",37.766920,-122.396027
6,Big Daddy's Antiques,"[{'id': '4bf58dd8d48988d116951735', 'name': 'A...",37.765015,-122.399497
7,Trimark Economy Restaurant Fixtures,"[{'id': '4bf58dd8d48988d1ff941735', 'name': 'M...",37.767543,-122.397641
8,Intelligentsia Roasting Works,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",37.763744,-122.395227
9,Seven Stills,"[{'id': '4e0e22f5a56208c4ea9a85a0', 'name': 'D...",37.768620,-122.399168


In [34]:
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Boba Guys,Bubble Tea Shop,37.766448,-122.397042
1,Bottom of the Hill,Rock Club,37.765116,-122.396218
2,Fitness Urbano,Gym / Fitness Center,37.765684,-122.397009
3,Pawtrero Hill Bathhouse and Feed Company,Pet Store,37.764140,-122.394500
4,UCSF Bakar Fitness & Rec Center,Gym,37.768146,-122.393290
5,Daggett Plaza,Park,37.766920,-122.396027
6,Big Daddy's Antiques,Antique Shop,37.765015,-122.399497
7,Trimark Economy Restaurant Fixtures,Miscellaneous Shop,37.767543,-122.397641
8,Intelligentsia Roasting Works,Office,37.763744,-122.395227
9,Seven Stills,Distillery,37.768620,-122.399168



# 10. We retrieve lists of venues in each San Francisco Zip code.

In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        venue_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in venue_results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
sanfran_venues = getNearbyVenues(names = df_merged['PostalCode'],
                                   latitudes = df_merged['Latitude'],
                                   longitudes = df_merged['Longitude']
                                  )



94102
94103
94104
94105
94107
94108
94109
94110
94111
94112
94114
94115
94116
94117
94118
94121
94122
94123
94124
94127
94129
94130
94131
94132
94133
94134


In [37]:
print(sanfran_venues.shape)
sanfran_venues.head(70)

(1417, 7)


,PostalCode,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,94102,37.779329,-122.41915,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
1,94102,37.779329,-122.41915,War Memorial Opera House,37.778601,-122.420816,Opera House
2,94102,37.779329,-122.41915,San Francisco Ballet,37.778580,-122.420798,Dance Studio
3,94102,37.779329,-122.41915,Herbst Theater,37.779548,-122.420953,Concert Hall
4,94102,37.779329,-122.41915,War Memorial Court,37.779042,-122.420971,Park
5,94102,37.779329,-122.41915,Asian Art Museum,37.780178,-122.416505,Art Museum
6,94102,37.779329,-122.41915,Philz Coffee,37.781266,-122.416901,Coffee Shop
7,94102,37.779329,-122.41915,"Books, Inc.",37.781614,-122.420531,Bookstore
8,94102,37.779329,-122.41915,Sydney Goldstein Theater,37.777031,-122.420979,Performing Arts Venue
9,94102,37.779329,-122.41915,Whitechapel,37.782230,-122.418884,Cocktail Bar


In [38]:
sanfran_venues.groupby('PostalCode').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode,,,,,,
94102,90,90,90,90,90,90
94103,77,77,77,77,77,77
94104,100,100,100,100,100,100
94105,81,81,81,81,81,81
94107,28,28,28,28,28,28
94108,93,93,93,93,93,93
94109,85,85,85,85,85,85
94110,47,47,47,47,47,47
94111,100,100,100,100,100,100


In [39]:
print('There are {} unique categories.'.format(len(sanfran_venues['Venue Category'].unique())))

There are 259 unique categories.


In [40]:
sanfran_onehot = pd.get_dummies(sanfran_venues[['Venue Category']], prefix="", prefix_sep="")

sanfran_onehot['PostalCode'] = sanfran_venues['PostalCode'] 
fixed_columns = [sanfran_onehot.columns[-1]] + list(sanfran_onehot.columns[:-1])
sanfran_onehot = sanfran_onehot[fixed_columns]
sanfran_onehot.head(70)


,PostalCode,ATM,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Outdoor Sculpture,Park,Parking,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Sicilian Restaurant,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,94102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [41]:
sanfran_grouped = sanfran_onehot.groupby('PostalCode').mean().reset_index()
sanfran_grouped

,PostalCode,ATM,Acai House,Accessories Store,Adult Boutique,African Restaurant,Alternative Healer,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Bike Rental / Bike Share,Bike Shop,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Café,Camera Store,Candy Store,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Credit Union,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,Empanada Restaurant,Entertainment Service,Ethiopian Restaurant,Event Space,Exhibit,Eye Doctor,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hill,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Opera House,Optical Shop,Outdoor Sculpture,Park,Parking,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Restaurant,Road,Rock Club,Roof Deck,Rugby Pitch,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Sicilian Restaurant,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Taiwanese Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Trattoria/Osteria,Tunnel,Turkish Restaurant,Tuscan Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,94102,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.011111,0.000000,0.00,0.000000,0.011111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.022222,0.011111,0.00,0.000000,0.022222,0.000000,0.00,0.000000,0.011111,0.022222,0.000000,0.00,0.000000,0.000000,0.000000,0.011111,0.00,0.011111,0.000000,0.000000,0.0,0.000000,0.000000

In [42]:
sanfran_grouped.PostalCode = sanfran_grouped.PostalCode.astype(str)

# 11. Identify the most common venues for each Zip code.

In [43]:
num_top_venues = 5
for code in sanfran_grouped['PostalCode']:
    print("----"+code+"----")
    temp = sanfran_grouped[sanfran_grouped['PostalCode'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----94102----
                           venue  freq
0                    Coffee Shop  0.06
1                          Hotel  0.04
2                           Café  0.04
3              French Restaurant  0.03
4  Vegetarian / Vegan Restaurant  0.03


----94103----
             venue  freq
0        Nightclub  0.09
1          Gay Bar  0.05
2     Cocktail Bar  0.05
3              Bar  0.04
4  Thai Restaurant  0.04


----94104----
                 venue  freq
0          Coffee Shop  0.07
1           Food Truck  0.05
2          Men's Store  0.04
3  Japanese Restaurant  0.04
4       Sandwich Place  0.03


----94105----
                  venue  freq
0           Coffee Shop  0.16
1            Food Truck  0.05
2                  Café  0.05
3                   Gym  0.04
4  Gym / Fitness Center  0.04


----94107----
            venue  freq
0            Café  0.07
1       Wine Shop  0.07
2            Park  0.07
3  Breakfast Spot  0.07
4            Pool  0.04


----94108----
                venue  f

In [44]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
sanfran_grouped.PostalCode = sanfran_grouped.PostalCode.astype(int)

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['PostalCode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = sanfran_grouped['PostalCode']

for ind in np.arange(sanfran_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sanfran_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,PostalCode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94102,Coffee Shop,Hotel,Café,Wine Bar,Vegetarian / Vegan Restaurant,French Restaurant,Boutique,Bakery,Optical Shop,Beer Bar
1,94103,Nightclub,Cocktail Bar,Gay Bar,Food Truck,Motorcycle Shop,Bar,Thai Restaurant,Cosmetics Shop,Furniture / Home Store,Sushi Restaurant
2,94104,Coffee Shop,Food Truck,Men's Store,Japanese Restaurant,Sandwich Place,Sushi Restaurant,Gym,Hotel,Juice Bar,Salad Place
3,94105,Coffee Shop,Food Truck,Café,Sandwich Place,Art Gallery,Gym / Fitness Center,Gym,Salad Place,New American Restaurant,Lounge
4,94107,Wine Shop,Breakfast Spot,Park,Café,Peruvian Restaurant,Pool,Bookstore,Coffee Shop,French Restaurant,Rock Club
5,94108,Hotel,Bakery,Coffee Shop,Chinese Restaurant,Men's Store,Cocktail Bar,Gym,French Restaurant,Sushi Restaurant,Szechuan Restaurant
6,94109,Grocery Store,Gym / Fitness Center,Deli / Bodega,Sushi Restaurant,Wine Bar,Vietnamese Restaurant,Pet Store,Gym,Thai Restaurant,Massage Studio
7,94110,Mexican Restaurant,Pizza Place,Coffee Shop,Park,Dive Bar,Gym / Fitness Center,Grocery Store,Juice Bar,Peruvian Restaurant,South American Restaurant
8,94111,Café,Food Truck,Scenic Lookout,Men's Store,Italian Restaurant,Cosmetics Shop,Coffee Shop,Park,New American Restaurant,Wine Bar
9,94112,Pizza Place,Bus Station,Mexican Restaurant,Vietnamese Restaurant,Sandwich Place,Flower Shop,Liquor Store,Gas Station,Fried Chicken Joint,Metro Station


# 12. Identify clusters within the data.

In [46]:
from sklearn.cluster import KMeans

kclusters = 5

sanfran_grouped_clustering = sanfran_grouped.drop('PostalCode', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(sanfran_grouped_clustering)

kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 4,
       3, 0, 0, 2], dtype=int32)

In [47]:
sanfran_merged = df_merged[0:47]

sanfran_merged['Cluster Labels'] = kmeans.labels_

sanfran_merged = sanfran_merged.join(neighborhoods_venues_sorted.set_index('PostalCode'), on='PostalCode')

sanfran_merged.head(47)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,PostalCode,City,Latitude,Longitude,Population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,94102,San Francisco,37.779329,-122.41915,San Francisco,0,Coffee Shop,Hotel,Café,Wine Bar,Vegetarian / Vegan Restaurant,French Restaurant,Boutique,Bakery,Optical Shop,Beer Bar
2,94103,San Francisco,37.772329,-122.41087,San Francisco,0,Nightclub,Cocktail Bar,Gay Bar,Food Truck,Motorcycle Shop,Bar,Thai Restaurant,Cosmetics Shop,Furniture / Home Store,Sushi Restaurant
3,94104,San Francisco,37.791728,-122.40190,San Francisco,0,Coffee Shop,Food Truck,Men's Store,Japanese Restaurant,Sandwich Place,Sushi Restaurant,Gym,Hotel,Juice Bar,Salad Place
4,94105,San Francisco,37.789228,-122.39570,San Francisco,0,Coffee Shop,Food Truck,Café,Sandwich Place,Art Gallery,Gym / Fitness Center,Gym,Salad Place,New American Restaurant,Lounge
5,94107,San Francisco,37.766529,-122.39577,San Francisco,0,Wine Shop,Breakfast Spot,Park,Café,Peruvian Restaurant,Pool,Bookstore,Coffee Shop,French Restaurant,Rock Club
6,94108,San Francisco,37.792678,-122.40793,San Francisco,0,Hotel,Bakery,Coffee Shop,Chinese Restaurant,Men's Store,Cocktail Bar,Gym,French Restaurant,Sushi Restaurant,Szechuan Restaurant
7,94109,San Francisco,37.792778,-122.42188,San Francisco,0,Grocery Store,Gym / Fitness Center,Deli / Bodega,Sushi Restaurant,Wine Bar,Vietnamese Restaurant,Pet Store,Gym,Thai Restaurant,Massage Studio
8,94110,San Francisco,37.748730,-122.41545,San Francisco,0,Mexican Restaurant,Pizza Place,Coffee Shop,Park,Dive Bar,Gym / Fitness Center,Grocery Store,Juice Bar,Peruvian Restaurant,South American Restaurant
9,94111,San Francisco,37.798228,-122.40027,San Francisco,0,Café,Food Truck,Scenic Lookout,Men's Store,Italian Restaurant,Cosmetics Shop,Coffee Shop,Park,New American Restaurant,Wine Bar
10,94112,San Francisco,37.720931,-122.44241,San Francisco,0,Pizza Place,Bus Station,Mexican Restaurant,Vietnamese Restaurant,Sandwich Place,Flower Shop,Liquor Store,Gas Station,Fried Chicken Joint,Metro Station


# 13. Generate map with clusters superimposed.

In [48]:
map_clusters = folium.Map(location=[37.801878, -122.41018], zoom_start=11)

x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)

markers_colors = []
for lat, lon, nei , cluster in zip(sanfran_merged['Latitude'], sanfran_merged['Longitude'], sanfran_merged['PostalCode'], sanfran_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


In [49]:
sanfran_merged.loc[sanfran_merged['Cluster Labels'] == 0, sanfran_merged.columns[[0] + list(range(5, sanfran_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,94102,0,Coffee Shop,Hotel,Café,Wine Bar,Vegetarian / Vegan Restaurant,French Restaurant,Boutique,Bakery,Optical Shop,Beer Bar
2,94103,0,Nightclub,Cocktail Bar,Gay Bar,Food Truck,Motorcycle Shop,Bar,Thai Restaurant,Cosmetics Shop,Furniture / Home Store,Sushi Restaurant
3,94104,0,Coffee Shop,Food Truck,Men's Store,Japanese Restaurant,Sandwich Place,Sushi Restaurant,Gym,Hotel,Juice Bar,Salad Place
4,94105,0,Coffee Shop,Food Truck,Café,Sandwich Place,Art Gallery,Gym / Fitness Center,Gym,Salad Place,New American Restaurant,Lounge
5,94107,0,Wine Shop,Breakfast Spot,Park,Café,Peruvian Restaurant,Pool,Bookstore,Coffee Shop,French Restaurant,Rock Club
6,94108,0,Hotel,Bakery,Coffee Shop,Chinese Restaurant,Men's Store,Cocktail Bar,Gym,French Restaurant,Sushi Restaurant,Szechuan Restaurant
7,94109,0,Grocery Store,Gym / Fitness Center,Deli / Bodega,Sushi Restaurant,Wine Bar,Vietnamese Restaurant,Pet Store,Gym,Thai Restaurant,Massage Studio
8,94110,0,Mexican Restaurant,Pizza Place,Coffee Shop,Park,Dive Bar,Gym / Fitness Center,Grocery Store,Juice Bar,Peruvian Restaurant,South American Restaurant
9,94111,0,Café,Food Truck,Scenic Lookout,Men's Store,Italian Restaurant,Cosmetics Shop,Coffee Shop,Park,New American Restaurant,Wine Bar
10,94112,0,Pizza Place,Bus Station,Mexican Restaurant,Vietnamese Restaurant,Sandwich Place,Flower Shop,Liquor Store,Gas Station,Fried Chicken Joint,Metro Station


In [50]:
sanfran_merged.loc[sanfran_merged['Cluster Labels'] == 1, sanfran_merged.columns[[0] + list(range(5, sanfran_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,94127,1,Trail,Bus Line,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flower Shop


In [51]:
sanfran_merged.loc[sanfran_merged['Cluster Labels'] == 2, sanfran_merged.columns[[0] + list(range(5, sanfran_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,94134,2,Baseball Field,Garden,Trail,Park,Yoga Studio,Filipino Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market


In [52]:
sanfran_merged.loc[sanfran_merged['Cluster Labels'] == 3, sanfran_merged.columns[[0] + list(range(5, sanfran_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,94131,3,Park,Trail,Grocery Store,Shopping Mall,Salon / Barbershop,Coffee Shop,Playground,Pharmacy,Dim Sum Restaurant,Dive Bar


In [53]:
sanfran_merged.loc[sanfran_merged['Cluster Labels'] == 4, sanfran_merged.columns[[0] + list(range(5, sanfran_merged.shape[1]))]]

,PostalCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,94130,4,Grocery Store,Music Venue,Flea Market,Harbor / Marina,Athletics & Sports,History Museum,Fried Chicken Joint,Brewery,Rugby Pitch,Flower Shop
